In [2]:
import pandas as pd
import yfinance as yf

from google.cloud import bigquery
from google.oauth2 import service_account 

#Authentication
key_path = "/home/om/Documents/Big_query/ncca-361107-9f401816514a.json" ##Replace with path to your service token file
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials,
project=credentials.project_id)

In [3]:
get_tickers = pd.read_csv("https://raw.githubusercontent.com/datasets/s-and-p-500-companies-financials/master/data/constituents.csv")
stock_dat = yf.download(get_tickers.Symbol.head(2).tolist(),period="2d",actions=False,group_by=None)


[*********************100%***********************]  2 of 2 completed


In [4]:
stock_dat

MMM                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2022-09-01  124.300003  126.389999  123.669998  125.629997  125.629997   
2022-09-02  126.029999  126.370003  121.150002  121.650002  121.650002   

                            AOS                                          \
              Volume       Open       High    Low      Close  Adj Close   
Date                                                                      
2022-09-01  31014400  56.049999  56.459999  55.68  56.419998  56.419998   
2022-09-02  12320600  57.230000  57.610001  56.00  56.270000  56.270000   

                    
            Volume  
Date                
2022-09-01  836400  
2022-09-02  678100

In [3]:
# Formating Data 

In [5]:
long_form = stock_dat.reset_index().melt('Date', var_name=['Ticker', 'var'])
long_form = long_form.pivot_table(index=['Date', 'Ticker'], columns='var', values='value').reset_index()


In [6]:
long_form

var,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2022-09-01,AOS,56.419998,56.419998,56.459999,55.680000,56.049999,836400.0
1,2022-09-01,MMM,125.629997,125.629997,126.389999,123.669998,124.300003,31014400.0
2,2022-09-02,AOS,56.270000,56.270000,57.610001,56.000000,57.230000,678100.0
3,2022-09-02,MMM,121.650002,121.650002,126.370003,121.150002,126.029999,12320600.0


In [5]:
# Processing and Cleaning Data to Make it More Readable and Easy To Analyse

In [ ]:
long_form["Change"] = long_form.groupby("Ticker").Close.diff()
long_form["Pct_Change"] = long_form.groupby("Ticker").Close.pct_change() * 100
long_form = long_form[long_form ["Date"] == long_form["Date"].iloc[long_form.shape[0]-1]]
long_form = long_form.rename(columns={"Adj Close":"Adj_Close"})
long_form["Date"] = long_form['Date'].dt.strftime('%Y-%m-%d') 


In [6]:
long_form

var,Date,Ticker,Adj_Close,Close,High,Low,Open,Volume,Change,Pct_Change
5,2022-09-02,ABBV,136.279999,136.279999,139.399994,135.809998,138.660004,3598800.0,-2.169998,-1.567352
6,2022-09-02,ABT,102.500000,102.500000,105.650002,101.980003,105.449997,4103200.0,-2.339996,-2.231969
7,2022-09-02,ACN,284.070007,284.070007,293.160004,282.820007,292.570007,1889400.0,-4.720001,-1.634406
8,2022-09-02,AOS,56.270000,56.270000,57.610001,56.000000,57.230000,678100.0,-0.149998,-0.265859
9,2022-09-02,MMM,121.650002,121.650002,126.370003,121.150002,126.029999,12320600.0,-3.979996,-3.168030


In [7]:
table = 'ncca-361107.stocks.stocks_data' ##Replace with your destination_name.dataset.table_name
job = client.load_table_from_dataframe(long_form, table)

In [ ]:
#Uploading Finale Data to Our Big Quetry Table
#Data can be viewwed As Follows-----------------------


In [38]:
query_data = 'SELECT * FROM ncca-361107.stocks.stocks_data limit 5' ##Replace with your destination_name.dataset.table_name
client.query(query_data).result().to_dataframe()

,Date,Ticker,Adj_Close,Close,High,Low,Open,Volume,Change,Pct_Change
0,2022-09-02,ABBV,136.279999,136.279999,139.399994,135.809998,138.660004,3598800.0,-2.169998,-1.567352
1,2022-09-02,ABT,102.500000,102.500000,105.650002,101.980003,105.449997,4103200.0,-2.339996,-2.231969
2,2022-09-02,ACN,284.070007,284.070007,293.160004,282.820007,292.570007,1889400.0,-4.720001,-1.634406
3,2022-09-02,AOS,56.270000,56.270000,57.610001,56.000000,57.230000,678100.0,-0.149998,-0.265859
4,2022-09-02,MMM,121.650002,121.650002,126.370003,121.150002,126.029999,12320600.0,-3.979996,-3.168030
